### Cargamos las librerias

In [ ]:
import pandas as pd 
import numpy as np
import ast
import json
import emoji
import re
import nltk
# Esto sirve para configurar NLTK. La primera vez puede tardar un poco
nltk.download('punkt')
nltk.download('stopwords')

## Cargamos la data

Para cargar nuestra data, seguimos un proceso distinto al habitual debido a que está en formato JSON y contiene anidaciones.

In [ ]:
rows = []

with open ('australian_user_reviews.json', encoding='utf-8') as f:
    for line in f.readlines():
        rows.append(ast.literal_eval(line))

df_reviews = pd.DataFrame(rows)

In [ ]:
rows = []

with open ('australian_users_items.json', encoding='utf-8') as f:
    for line in f.readlines():
        rows.append(ast.literal_eval(line))

df_items = pd.DataFrame(rows)

In [ ]:
filas = []

with open("output_steam_games.json", encoding='utf-8') as file:
    for line in file.readlines():
        filas.append(json.loads((line))) # Como los tipo de valores no eran strings que es el tipo esperado por ast.litreal_eval, usamos json.loads

df_games = pd.DataFrame(filas)

Asi se veria nuestro DataFrame df_reviews. Observar que la columna reviews tiene una lista de diccionario por lo cual debemos de realizar el siguiente proceso para poder desanidar esta columna 

In [ ]:
df_reviews.head()

desanidamos el dataFrame df_items 

In [ ]:
data_desanidada = []

for index, row in df_items.iterrows():
    user_id = row['user_id']
    items_count = row['items_count']
    steam_id = row['steam_id']
    user_url = row['user_url']
    items = row['items']
    
    for i in items:   
        new_row = {
        'user_id': user_id,
        'items_count': items_count,
        'steam_id' : steam_id,
        'user_url' : user_url,
        'item_id': i.get('item_id', ''),
        'item_name': i.get('item_name', ''),
        'playtime_forever': i.get('playtime_forever', ''),
        'playtime_2weeks': i.get('playtime_2weeks', '')
        }
        
        data_desanidada.append(new_row)

df_items_desanidada = pd.DataFrame(data_desanidada)

Desanidamos el dataFrame df_reviews

In [ ]:
data_desanidada = []

for index, row in df_reviews.iterrows():
    user_id = row['user_id']
    user_url = row['user_url']
    
    reviews= row['reviews']
    
    
    for i in reviews:   
        new_row = {
        'user_id': user_id,
        'user_url':user_url,
        
        'funny': i.get('funny', ''),
        'posted': i.get('posted', ''),
        'last_edited': i.get('last_edited', ''),
        'item_id': i.get('item_id', ''),
        'helpful': i.get('helpful', ''),
        'recommend': i.get('recommend', ''),
        'review': i.get('review', '')
        }
        
        data_desanidada.append(new_row)

df_reviews_desanidada=pd.DataFrame(data_desanidada)

## Analisis de sentimiento
Este analisis lo vamos a realizar para nuestro df_reviews_desanidada el cual tiene las reviews hecha por los usuarios de los video juegos. Para lo cual es necesario que nuestra columna reviews no tenga valores nulos o vacios para lo cual aplicamos una mascara que filtre todos los valores execto los vacios.

In [ ]:
df_reviews_desanidada=df_reviews_desanidada[df_reviews_desanidada['review']!=""]

### Filtrado de Emojis en la Columna de Review

Nuestra columna de "review" contiene comentarios con emojis, por lo que es necesario filtrar estos emojis para realizar un tratamiento de datos más efectivo. Esto nos permitirá obtener información adicional para nuestro análisis de sentimiento. Para llevar a cabo este proceso, vamos a utilizar una librería llamada `emoji`.

Para comenzar, crearemos una función llamada `emojis_count(x)`. Esta función tomará como parámetro (x) un texto y devolverá una lista con los emojis presentes en dicho texto.


In [ ]:
def emojis_count(x):
    emojis = []
    emojis_encontrados = emoji.emoji_list(x)
    if emojis_encontrados == '':
        return ''
    for e in emojis_encontrados:
        emojis.append(e['emoji'])
    return emojis

Vamos a emplear una función lambda para ingresar de manera individual cada uno de los textos de la columna "review" en la función que creamos en el paso anterior. Esta acción nos devolverá una lista; en caso de no encontrar emojis, la lista resultante estará vacía.

In [ ]:
emojis = df_reviews_desanidada['review'].apply(lambda x: emojis_count(x))
emojis

La siguiente función, `obtener_emojis_unicos(emojis)`, tiene como objetivo devolver una lista de emojis únicos. Es decir, eliminará los emojis repetidos para obtener una lista que contenga cada emoji una sola vez.

In [ ]:
def obtener_emojis_unicos(emojis):
    return list(set(emojis))

En este siguiente paso, realizamos dos procesos:

1. **Filtramos las listas vacías** obtenidas en el paso anterior.
2. Empleamos la función creada anteriormente, `obtener_emojis_unicos(emojis)`, para **eliminar los emojis repetidos** de la lista resultante.

In [ ]:
emojis = emojis[emojis.apply(lambda x: len(x) > 0)]
emojis = emojis.apply(obtener_emojis_unicos)

Eliminamos los emojis duplicados

In [ ]:
emojis.drop_duplicates(inplace=True)
emojis.count()

Aquí se encuentran todos los emojis únicos que se hallaron en la columna de "review".

In [ ]:
emojis

### Asignación de Palabras a Emojis y Signos de Puntuación

Creamos una lista de diccionarios en la que cada emoji encontrado tiene asignada una palabra específica para su reemplazo. Además, solicitamos a ChatGPT una lista de emojis correspondientes a signos de puntuación (:D, :p, :(), <3) a los cuales también asignamos una palabra para su representación.

In [ ]:
emoji_reemplazos = {
    '✔': 'Approved ',
    '💯': 'Perfect ',
    '♀': 'Woman ',
    '❤': 'Love ',
    '💋': 'Kiss ',
    '💀': 'Skull ',
    '™': 'Trademark ',
    '®': 'Registered Trademark ',
    '©': 'Copyright ',
    '☺': 'Smile ',
    '♥': 'Heart ',
    '👌': 'Okay ',
    '👀': 'Eye ',
    '⌛': 'Hourglass ',
    '☹': 'Sad ',
    '☑': 'Checked ',
    '😊': 'Happiness ',
    '😄': 'Happiness ',
    '◀': 'Left ',
    ':D': 'laugh ',
    ':3': 'smile',
    'xD': 'laughloud',
    'ô•': 'oh',
    ':c': 'adictive',
    ':)': 'happy',
    ':p': 'boring',
    '^^': 'good',
    'D=': 'pain',
    ':(': 'sad',
    'xd': 'happiness',
    '=P': 'crazy',
    '<3': 'heart',
    ';>': 'blink',
    '(Y)': 'thumbs up',
    '---': 'enjoyable',
    ',3': 'amazing',
    'xt': 'unwanted',
    '-' : 'recommended',
    ':}': 'good',
    'c:': 'happy',
    ':v': 'want more',
    'x3': 'cute face',
    'XD': 'adictive',
    '(y)': 'thumbs up',
    ':P': 'bad',
    ';0': 'laugh',
    '=)': 'happy',
    'gg': 'great game',
    'GG': 'great game',
    ';D': 'great game',
    ':c': 'sad'
}

Creamos una función que reemplaza cada uno de los emojis por las palabras asignadas previamente.

In [ ]:
def replace_emojis(text):
    for emoji, replacement in emoji_reemplazos.items():
        text = text.replace(emoji, replacement)
    return text

Aplicamos la función `replace_emojis(text)` a nuestra columna de "review".

In [ ]:
df_reviews_desanidada['review']=df_reviews_desanidada['review'].apply(replace_emojis)

## Análisis de Sentimientos

En este análisis, vamos a utilizar la librería NLTK para determinar si las reviews contienen comentarios positivos, negativos o neutros. Para llevar a cabo este análisis, utilizamos las siguientes librerías.

In [ ]:
# Importamos esta libreria que nos permite reemplzar caracteres
import re

# Importamos la función que nos permite Stemmizar de nltk y definimos el stemmer
from nltk.stem import PorterStemmer
stemmer = PorterStemmer()

# Traemos nuevamente las stopwords
stopwords = nltk.corpus.stopwords.words('english')

La siguiente función realiza una serie de procesamientos sobre la columna 'review' de un DataFrame. Primero, itera a través de cada fila del DataFrame y realiza las siguientes acciones para cada 'review':

1. Reemplaza los caracteres que no son letras por espacios.
2. Convierte todas las letras a minúsculas.
3. Tokeniza el texto para separar las palabras.
4. Elimina las palabras con menos de un carácter.
5. Remueve las palabras de parada (stopwords).
6. Aplica un proceso de stemming para buscar la raíz de las palabras.
7. Finalmente, une nuevamente las palabras para reconstruir la review procesada y la agrega a una lista llamada `review_list`.

Luego, crea una nueva columna en el DataFrame llamada 'review_normalizado' que contiene las reviews procesadas. Por último, imprime el DataFrame con las reviews procesadas.

In [ ]:
review_list = []

# Itera a través de las filas del DataFrame y su columna 'review'
for index, row in df_reviews_desanidada.iterrows():
    review = row['review']
    
    # Reemplazamos los caracteres que no sean letras por espacios
    review = re.sub("[^a-zA-Z]", " ", str(review))
    
    # Convertimos todo a minúsculas
    reviewr = review.lower()
    
    # Tokenizamos para separar las palabras
    review = nltk.word_tokenize(review)
    
    # Eliminamos las palabras de menos de 1 caracter
    review = [palabra for palabra in review if len(palabra) > 1]
    
    # Sacamos las Stopwords
    review = [palabra for palabra in review if not palabra in stopwords]
    
    # Aplicamos la función para buscar la raíz de las palabras
    review = [stemmer.stem(palabra) for palabra in review]
    
    # Por último, volvemos a unir el titular
    review = " ".join(review)
    
    # Agregamos el titular procesado a la lista
    review_list.append(review)

# Crear una nueva columna en el DataFrame con los titulares procesados
df_reviews_desanidada['review_normalizado'] = review_list

# Imprimir el DataFrame con los titulares procesados
print(df_reviews_desanidada)

## Procesamiento de Sentimientos

Importamos la librería NLTK para analizar los sentimientos de nuestra columna de reviews.

In [ ]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

### Opcional: Código Alternativo en Caso de Error al Cargar el Archivo Vader con NLTK

En caso de que la librería NLTK no pueda cargar el archivo para Vader, se puede usar el siguiente código como alternativa.

In [ ]:
import nltk
nltk.download('vader_lexicon')

### Ejemplo de Análisis de Sentimientos con NLTK

A continuación, presentaremos un breve ejemplo para mostrar cómo la librería NLTK realiza el análisis de sentimientos.

In [ ]:
# Texto a analizar
x="it is a charming and beautiful produc"

Vamos a instanciar el analizador de texto y realizar el análisis del texto previamente mencionado.

In [ ]:
sid = SentimentIntensityAnalyzer()  # Instanciamos el analizados de texto
resultado= sid.polarity_scores(x) # analizamos el texto
resultado

Observa que la salida del análisis  incluye la probabilidad de que el texto tenga sentimiento positivo, negativo o neutro. En este caso, el sentimiento positivo tiene una probabilidad de 0.658.

También podemos analizar la información proporcionada por el valor `compound`, el cual indica que si es mayor a 0, el texto tiene un sentimiento positivo; si es menor a 0, el sentimiento es negativo; y si es igual a 0, el sentimiento del texto es neutro.

## Retomando el Análisis de Sentimiento de la Columna 'reviews'

Vamos a aplicar una función lambda a cada elemento de la columna 'review_normalizado' para realizar el análisis de sentimiento. Esto nos devolverá únicamente el valor `compound`, el cual será almacenado en una nueva columna llamada 'sentiment_analysis'.

In [ ]:
df_reviews_desanidada['sentiment_analysis']=df_reviews_desanidada["review_normalizado"].apply(lambda i: sid.polarity_scores(i)['compound'])



A continuación, presentaremos un breve ejemplo para mostrar cómo la librería NLTK realiza el análisis de sentimientos. **Es importante considerar** el orden en el que se realizan los reemplazos, ya que estos podrían intercambiar los valores.

### Reemplazo de Valores en la Columna 'sentiment_analysis'

Vamos a reemplazar los valores de la columna 'sentiment_analysis' con los parámetros indicados en el archivo README del proyecto. Es esencial considerar el orden en el que se efectúan los reemplazos para evitar intercambios de valores.

In [ ]:
df_reviews_desanidada.loc[df_reviews_desanidada['sentiment_analysis']>0,'sentiment_analysis']=2
df_reviews_desanidada.loc[df_reviews_desanidada['sentiment_analysis']==0,'sentiment_analysis']=1
df_reviews_desanidada.loc[df_reviews_desanidada['sentiment_analysis']<0,'sentiment_analysis']=0

### Conversión de la Columna 'recommden' a Dummy

Vamos a convertir la columna 'recommden' a una variable dummy.

In [ ]:
dummy_recommend=pd.get_dummies(df_reviews_desanidada["recommend"], prefix="recommend").astype(int)
df_reviews_desanidada=pd.concat([df_reviews_desanidada,dummy_recommend], axis=1)

## Exportamos la Data

In [ ]:
df_reviews_desanidada.to_csv('reviews.csv', index=False)
df_items_desanidada.to_csv('items.csv', index=False)
df_games.to_csv('games.csv', index=False)